<a href="https://colab.research.google.com/github/michalis0/DataScience_and_MachineLearning/blob/master/06-recommender-systems/Week_06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.metrics.pairwise import cosine_similarity


# Recommender Systems


<img src='https://imgs.xkcd.com/comics/star_ratings.png' width="300">

Source: [xkcd 1908](https://xkcd.com/1098/)

### Goal
In this exercise, we will be proceeding in two stages.
1. The first stage is where we get into the details of how to build our own recommender system to recommend movies to users.
2. In the second stage, we will be an existing library, specialized for recommender systems, which provides more powerful options. We will be testing it on the task of recommending jokes to users.

### What you are learning in this exercise:
1. Getting familiar with item-based collaborative filtering and user-based collaborative filtering.
2. Getting familiar with an existing library for recommender systems.

### Content

The goal of this walkthrough is to guide you through implementing and evaluating a basic recommender system. A [recommender system](https://en.wikipedia.org/wiki/Recommender_system) is a type of information filtering system designed to predict a user’s preference for an item, often used in commercial applications to suggest products, movies, or other items.

In this notebook, we will cover:

- [Task 1: Exploring the MovieLens Dataset with Implicit Feedback](#Task-1-Exploring-the-MovieLens-Dataset-with-Implicit-Feedback)
  - [Step 1: Load the Data](#Step-1-Load-the-Data)
  - [Step 2: Check the Number of Interactions, Users, and Movies](#Step-2-Check-the-Number-of-Interactions-Users-and-Movies)
  - [Step 3: Split the Data into Training and Test Sets](#Step-3-Split-the-Data-into-Training-and-Test-Sets)
  
- [Task 2: Creating User-Item Matrices for Implicit Feedback](#Task-2-Creating-User-Item-Matrices-for-Implicit-Feedback)
  - [Step 1: Define the Function to Create the Data Matrix](#Step-1-Define-the-Function-to-Create-the-Data-Matrix)
  - [Step 2: Create the Training and Testing Matrices](#Step-2-Create-the-Training-and-Testing-Matrices)
  - [Step 3: Visualize the User-Item Interaction Matrices](#Step-3-Visualize-the-User-Item-Interaction-Matrices)

- [Task 3: Item-to-Item Collaborative Filtering with Implicit Feedback](#Task-3-Item-to-Item-Collaborative-Filtering-with-Implicit-Feedback)
  - [Step 1: Compute Item Similarity Matrix](#Step-1-Compute-Item-Similarity-Matrix)
  - [Step 2: Predict Positive Interactions Using Item Similarity](#Step-2-Predict-Positive-Interactions-Using-Item-Similarity)

- [Task 4: User-to-User Collaborative Filtering with Implicit Feedback](#Task-4-User-to-User-Collaborative-Filtering-with-Implicit-Feedback)
  - [Step 1: Compute User Similarity Matrix](#Step-1-Compute-User-Similarity-Matrix)
  - [Step 2: Predict Positive Interactions Using User Similarity](#Step-2-Predict-Positive-Interactions-Using-User-Similarity)

- [Task 5: Evaluating Our Recommenders](#Task-5-Evaluating-Our-Recommenders)
  - [Step 5.1: Precision@K and Recall@K](#Step-5.1-Precision@K-and-Recall@K)
  - [Step 5.2: Implement Precision@K and Recall@K](#Step-5.2-Implement-Precision@K-and-Recall@K)

- [Task 6: Show the Recommendations for a Specific User](#Task-6-Show-the-Recommendations-for-a-Specific-User)
  - [Step 6.1: Select a User and Get Recommendations](#Step-6.1-Select-a-User-and-Get-Recommendations)
  - [Step 6.2: Create Recommendation DataFrames with TMDb IDs](#Step-6.2-Create-Recommendation-DataFrames-with-TMDb-IDs)
  - [Step 6.3: Fetching and Displaying Movie Posters](#Step-6.3-Fetching-and-Displaying-Movie-Posters)

## Task 1: Exploring the MovieLens Dataset with Implicit Feedback




In this part, we’ll be using the [MovieLens dataset](https://grouplens.org/datasets/movielens/). ‌We will use the Movielens-small dataset that contains 100,000 interactions of users with different movies. An interaction of a user with a movie is when a user rates a movie with a value from 1 to 5. In this walkthrough we want to focus on **implicit feedback** recommendation scenario, i.e., we are interested in the items that each user liked, or interacted with. Therefore, we have provided you with a subset of interactions in the dataset that contains the ratings larger than or equal to 4.

#### Step 1: Load the Data

We’ll start by loading the datasets in the `data` folder. We have 3 datasets here:
- `interactions.csv`
- `movies.csv`
- `links.csv`


In [ ]:
# Load the datasets
interactions = pd.read_csv('https://raw.githubusercontent.com/michalis0/DataScience_and_MachineLearning/refs/heads/master/Labs/06-recommender-systems/data/interactions.csv')
movies = pd.read_csv("https://raw.githubusercontent.com/michalis0/DataScience_and_MachineLearning/refs/heads/master/Labs/06-recommender-systems/data/movies.csv")
links = pd.read_csv("https://raw.githubusercontent.com/michalis0/DataScience_and_MachineLearning/refs/heads/master/Labs/06-recommender-systems/data/links.csv")
# Display the first rows of each dataset
display(interactions.head())
display(movies.head())
display(links.head())


__Can you understand the information contained in each of the tables above?__


#### Step 2: Check the Number of interactions, users and movies

Let’s check how many unique users and items we have in this dataset.

In [ ]:
n_users = interactions.user_id.nunique()
n_items = interactions.movie_id.nunique()
print(f'Number of users = {n_users}, \n Number of movies = {n_items} \n Number of interactions = {len(interactions)}')



#### Step 3: Split the Data into Training and Test Sets

Splitting the data to training and test sets for recommendation systems is different from what you have seen in other classification/regression problems. In recommender systems, we want to make recommendations (predictions) for each user. Therefore, we need to make sure that we have training and test data for each user. In other words, we have to split the interactions of each user to training and test sets. For instance, if we want to do 80-20 splitting, we need to make sure that we have 80% of each user's interactions in the training set and the remaining 20% in the test set.

Moreover, to have a splitting that is more aligned with the reality, it is better to do the splitting based on the timestamp of the interactions (if the timestamp data is available). To do so, we have to sort the interactions of each user based on time, and pick the first 80% interactions in the training set and the last 20% interactions in the test set. This is more aligned with the reality as we always want to predict the future interactions of a user, that is, *the items that the user might like in the future*.

The figure below can help you understand better how this splitting is done.



<img src='https://raw.githubusercontent.com/michalis0/DataScience_and_MachineLearning/refs/heads/master/09-recommender-systems/data/image/train-test-split-reco.png' width="700">

In [ ]:
# let's first sort the interactions by user and time stamp
interactions = interactions.sort_values(["user_id", "timestamp"])
interactions.head(10)

Next we can use the percentage rank from pandas to get a proportional ranking of the timestamps for each user. Check the documentation on how to use the `rank` funciton with groupby in pandas (https://pandas.pydata.org/docs/reference/api/pandas.core.groupby.DataFrameGroupBy.rank.html#pandas.core.groupby.DataFrameGroupBy.rank).

In [ ]:
interactions["pct_rank"] = interactions.groupby("user_id")["timestamp"].rank(pct=True, method='dense')
interactions.reset_index(inplace=True, drop=True)
interactions.head(10)

Now all remains to do is to pick the first 80% of the interactions of each user in the training set and the rest in the test set. We can do so using the `pct_rank` column.

In [6]:
train_data = interactions[interactions["pct_rank"] < 0.8]
test_data = interactions[interactions["pct_rank"] >= 0.8]

In [ ]:
print("Training set size:", train_data.shape[0])
print("Testing set size:", test_data.shape[0])


## Task 2: Creating User-Item Matrices for Implicit Feedback


In [ ]:

print('number of users =', n_users, '| number of movies =', n_items)


#### Step 1: Define the Function to Create the Data Matrix

Next, we’ll define a function that creates the user-item data matrix. Each matrix cell will contain a 1 if there was an interaction and a 0 otherwise.


In [9]:
import numpy as np

# Define a function to create the data matrix
def create_data_matrix(data, n_users, n_items):
    """
    This function returns a numpy matrix with shape (n_users, n_items).
    Each entry is a binary value indicating positive interaction.
    """
    data_matrix = np.zeros((n_users, n_items))
    data_matrix[data["user_id"].values, data["movie_id"].values] = 1
    return data_matrix


#### Step 2: Create the Training and Testing Matrices

Now we can use the function to create matrices for both the training and testing data. Each cell in the matrix will show a 1 if there was a positive interaction in the training or testing data, and a 0 otherwise.

In [ ]:
# Create the training and testing matrices
train_data_matrix = create_data_matrix(train_data, n_users, n_items)
test_data_matrix = create_data_matrix(test_data, n_users, n_items)

# Display the matrices to understand their structure
print('train_data_matrix')
print(train_data_matrix)
print("number of non-zero values: ", np.sum(train_data_matrix))
print('test_data_matrix')
print(test_data_matrix)
print("number of non-zero values: ", np.sum(test_data_matrix))


#### Step 3: Visualize the User-Item Interaction Matrices

To better understand the distribution of  interactions, we’ll use a **heatmap** to visualize a subset of each user-item matrix. In the heatmap:
- Rows represent users, and columns represent items.
- Each cell shows whether there’s a positive interaction (`1`) or no interaction (`0`).
- A color gradient makes it easy to spot clusters of interactions.

We’ll visualize only a small portion of the interaction matrices (e.g., 50x50) to keep the display manageable.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Function to plot a heatmap for a subset of the user-item matrix
def plot_interaction_heatmap(data_matrix, title, num_users=50, num_items=50):
    """
    Plots a heatmap of a subset of the user-item interaction matrix.
    Parameters:
        data_matrix: The matrix to visualize.
        title: The title of the plot.
        num_users: The number of users to display (rows).
        num_items: The number of items to display (columns).
    """
    # Extract a subset of the matrix for visualization purposes
    matrix_subset = data_matrix[:num_users, :num_items]

    plt.figure(figsize=(10, 8))
    sns.heatmap(matrix_subset, cmap="YlGnBu", cbar=True, cbar_kws={'label': 'Interaction (1 = Positive, 0 = None)'})
    plt.xlabel('Item ID')
    plt.ylabel('User ID')
    plt.title(title)
    plt.show()

# Visualize a subset of the training and testing matrices
plot_interaction_heatmap(train_data_matrix, 'User-Item Interaction Matrix (Train Data)')



## Task 3: Item-to-Item Collaborative Filtering with Implicit Feedback

Now that we’ve prepared our data, our next task is to create a recommender system using **Item-to-Item Collaborative Filtering**. In this context, the recommendation translates to “Users who liked this item (movie) also liked …”.

Since we’re working with implicit feedback, the prediction formula will focus on the **likelihood of positive interactions** rather than ratings. This will predict items the user may interact with based on similarities to items they’ve previously liked.

The following formula shows how to compute the likelihood of interaction between a user $u$ and item $i$ using item-to-item colaborative filtering.

\begin{equation}
{p}_{u}(i) =  \frac{\sum\limits_{i' \in I} \text{sim}(i, i') \cdot {R}_{u}(i')}{\sum\limits_{i' \in I} \text{sim}(i, i')}
\end{equation}

In the above formula:


*   $P_u(i)$ is the likelihood of user $u$ interacting with item $i$. This is the value we want to compute.
*   $\text{sim}(i, i')$ is the cosine similarity between items $i$ and $i'$.
*   $R_u(i')$ is one if user $u$ has already interacted with item $i'$. Otherwise it is zero.
*   $I$ is the set of all items in the dataset.

The value $P_u(i)$ is expected to be between 0 and 1.



#### Step 1: Compute Item Similarity Matrix

As a first step, we’ll calculate the pairwise similarity matrix between all items using **cosine similarity**. This matrix will show how similar each item is to every other item based on the interaction patterns of users. The output will be an `n_items` by `n_items` symmetric 2D numpy matrix.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Compute the item-item similarity matrix
# Check sklearn documentation for cosine similarity to understand how we are
# using this function.
item_similarity = cosine_similarity(train_data_matrix.T)
print("Item-Item Similarity Matrix:")
print(item_similarity)
print(item_similarity.shape)


In [ ]:
# Function to visualize the item similarity matrix
def plot_item_similarity_heatmap(similarity_matrix, title, num_items=50):
    """
    Plots a heatmap of a subset of the item similarity matrix.
    Parameters:
        similarity_matrix: The item-item similarity matrix.
        title: The title of the plot.
        num_items: The number of items to display (both rows and columns).
    """
    # Extract a subset of the matrix for visualization
    matrix_subset = similarity_matrix[:num_items, :num_items]

    plt.figure(figsize=(10, 8))
    sns.heatmap(matrix_subset, cmap="coolwarm", cbar=True, cbar_kws={'label': 'Similarity Score'})
    plt.xlabel('Item ID')
    plt.ylabel('Item ID')
    plt.title(title)
    plt.show()

# Visualize a subset of the item similarity matrix
plot_item_similarity_heatmap(item_similarity, 'Item-Item Similarity Matrix')


Let's have a look at the 2 items that have a similarity of one. Do you think they are actually similar? What do you think has happened here?

In [ ]:
item_similarity[46, 37]

In [ ]:
# Print the item with ID 46
display(movies[movies['movie_id'] == 46])

# Print the item with ID 37
display(movies[movies['movie_id'] == 37])

#### Step 2: Predict Positive Interactions Using Item Similarity

Next, we’ll use the item similarity matrix to predict the likelihood of positive interactions for each user with each item.

In what follows, we create a function that implements the above formula for item-to-item cf recommendation.

In [ ]:
import numpy as np

# Define the function to predict interactions based on item similarity
def item_based_predict(interactions, similarity, epsilon=1e-9):
    """
    Predicts user-item interactions based on item-item similarity.
    Parameters:
        interactions (numpy array): The user-item interaction matrix.
        similarity (numpy array): The item-item similarity matrix.
        epsilon (float): Small constant added to the denominator to avoid division by zero.
    Returns:
        numpy array: The predicted interaction scores for each user-item pair.
    """
    # np.dot does the matrix multiplication. Here we are calculating the
    # weighted sum of interactions based on item similarity
    pred = similarity.dot(interactions.T) / (similarity.sum(axis=1)[:, np.newaxis] + epsilon)
    return pred.T  # Transpose to get users as rows and items as columns

# Calculate the item-based predictions for positive interactions
item_prediction = item_based_predict(train_data_matrix, item_similarity)
print("Predicted Interaction Matrix:")
print(item_prediction)
print(item_prediction.shape)

In [ ]:
# Function to visualize the predicted interaction matrix for a subset of users and items
def plot_interaction_prediction_heatmap(prediction_matrix, title, num_users=50, num_items=50):
    """
    Plots a heatmap of a subset of the user-item predicted interaction matrix.
    Parameters:
        prediction_matrix: The predicted interaction matrix.
        title: The title of the plot.
        num_users: The number of users to display (rows).
        num_items: The number of items to display (columns).
    """
    # Extract a subset of the matrix for visualization
    matrix_subset = prediction_matrix[:num_users, :num_items]

    plt.figure(figsize=(10, 8))
    sns.heatmap(matrix_subset, cmap="YlGnBu", cbar=True, cbar_kws={'label': 'Predicted Interaction Score'})
    plt.xlabel('Item ID')
    plt.ylabel('User ID')
    plt.title(title)
    plt.show()

# Visualize a subset of the predicted interaction matrix
plot_interaction_prediction_heatmap(item_prediction, 'User-Item Predicted Interaction Matrix')



In this setup:
- The `item_based_predict` function calculates a score for each user-item pair using the item-to-item colaborative filtering formula.
- The result is a prediction matrix where each cell indicates the predicted likelihood of an interaction between each user and item.

## Task 4: User-to-User Collaborative Filtering with Implicit Feedback

In this task, we’ll create a recommender system using **User-to-User Collaborative Filtering**. This approach translates to “Users who are similar to you also liked…”. We’ll predict the likelihood of a user interacting with an item based on the behavior of similar users.


#### Step 1: Compute User Similarity Matrix

To start, we’ll compute the similarity matrix between users using **cosine similarity**. This matrix shows how similar each user is to every other user based on their interaction patterns. This will create a `num_users * num_users` matrix.

In [ ]:
# Compute the user-user similarity matrix
user_similarity = cosine_similarity(train_data_matrix)
print("User-User Similarity Matrix:")
print(user_similarity)

# Check the shape as a sanity check
print("Shape of User Similarity Matrix:", user_similarity.shape)


In [ ]:
# Function to plot the user similarity matrix
def plot_user_similarity_heatmap(similarity_matrix, title, num_users=50):
    """
    Plots a heatmap of a subset of the user similarity matrix.
    Parameters:
        similarity_matrix: The user-user similarity matrix.
        title: The title of the plot.
        num_users: The number of users to display (both rows and columns).
    """
    # Extract a subset of the matrix for visualization
    matrix_subset = similarity_matrix[:num_users, :num_users]

    plt.figure(figsize=(10, 8))
    sns.heatmap(matrix_subset, cmap="coolwarm", cbar=True, cbar_kws={'label': 'Similarity Score'})
    plt.xlabel('User ID')
    plt.ylabel('User ID')
    plt.title(title)
    plt.show()

# Visualize a subset of the user similarity matrix
plot_user_similarity_heatmap(user_similarity, 'User-User Similarity Matrix')


### Step 2: Predict Positive Interactions Using User Similarity

Next, we’ll use the user similarity matrix to predict the likelihood of positive interactions for each user with each item. Since we’re working with implicit feedback, we’ll calculate a score for each item based on binary interaction values (1 for positive interactions, 0 for none).

The prediction formula is as follows:

\begin{equation}
{p}_{u}(i) = \frac{\sum\limits_{u' \in U} \text{sim}(u, u') \cdot {R}_{u'}(i)}{\sum\limits_{u' \in U} \text{sim}(u, u')}
\end{equation}


#### Explanation:
*   $P_u(i)$ is the likelihood of user $u$ interacting with item $i$. This is the value we want to compute.
*   $\text{sim}(u, u')$ is the cosine similarity between users $u$ and $u'$.
*   $R_u'(i)$ is one if user $u'$ has already interacted with item $i$. Otherwise it is zero.
*   $U$ is the set of all users in the dataset.


This formula calculates the likelihood of interaction between user $ u $ and item $ i $ based on interactions of similar users with that item. The weighted sum of interactions from similar users is normalized by the sum of the similarities.

- **`similarity.dot(interactions)`**: Calculates the weighted sum of interactions for each user, based on similar users’ interactions.
- **`np.abs(similarity).sum(axis=1)[:, np.newaxis]`**: The denominator normalizes the prediction by the sum of absolute similarities to avoid bias from users with more neighbors.
- **`epsilon`**: Prevents division by zero, which could result in `NaN` values.

In [ ]:
# Define the function to predict interactions based on user similarity
def user_based_predict(interactions, similarity, epsilon=1e-9):
    """
    Predicts user-item interactions based on user-user similarity.
    Parameters:
        interactions (numpy array): The user-item interaction matrix.
        similarity (numpy array): The user-user similarity matrix.
        epsilon (float): Small constant added to the denominator to avoid division by zero.
    Returns:
        numpy array: The predicted interaction scores for each user-item pair.
    """
    # Calculate the weighted sum of interactions based on user similarity
    pred = similarity.dot(interactions) / (np.abs(similarity).sum(axis=1)[:, np.newaxis] + epsilon)
    return pred

# Calculate the user-based predictions for positive interactions
user_prediction = user_based_predict(train_data_matrix, user_similarity)
print("Predicted Interaction Matrix (User-Based):")
print(user_prediction)
print(user_prediction.shape)


In [ ]:
# Function to visualize the predicted interaction matrix for a subset of users and items
def plot_interaction_prediction_heatmap(prediction_matrix, title, num_users=50, num_items=50):
    """
    Plots a heatmap of a subset of the user-item predicted interaction matrix.
    Parameters:
        prediction_matrix: The predicted interaction matrix.
        title: The title of the plot.
        num_users: The number of users to display (rows).
        num_items: The number of items to display (columns).
    """
    # Extract a subset of the matrix for visualization
    matrix_subset = prediction_matrix[:num_users, :num_items]

    plt.figure(figsize=(10, 8))
    sns.heatmap(matrix_subset, cmap="YlGnBu", cbar=True, cbar_kws={'label': 'Predicted Interaction Score'})
    plt.xlabel('Item ID')
    plt.ylabel('User ID')
    plt.title(title)
    plt.show()

# Visualize a subset of the predicted interaction matrix
plot_interaction_prediction_heatmap(user_prediction, 'User-Item Predicted Interaction Matrix (User-Based)')


## Task 5: Evaluating Our Recommenders

#### Ranking Accurarcy (Precision@K and Recall@K)

An implicit recommendation system is a ranking system which ranks items for each user based on their relevance to the user. Therefore, to evaluate such system, we’re interested in the quality of the top ranked recommendations rather than the actual predicted score for each recommendation. **Precision@K** and **Recall@K** focus on the top-K items recommended for each user, measuring relevance of the top-ranked recommended items to a user.

- **Precision@K**: Measures the proportion of relevant items in the top-K recommendations.
  
  \begin{equation}
  \text{Precision@K} = \frac{\text{Number of relevant items in top-K}}{K}
  \end{equation}

- **Recall@K**: Measures the proportion of actual relevant items that appear in the top-K recommendations.

  \begin{equation}
  \text{Recall@K} = \frac{\text{Number of relevant items in top-K}}{\text{Total number of relevant items}}
  \end{equation}

Recall@k and Precision@k are metrics that are computed per user. To have a single value to evaluate a recommender system as a whole, we can take the average of Recall@K and Precision@K among all of the users.

### Your Turn !

As an exercise, complete the function precision_recall_at_k below to calculate Precision@K and Recall@K for a given value of K. Follow the instructions provided in the function to fill in the missing parts.

In [22]:
import numpy as np

# TODO: Implement the precision_recall_at_k function
def precision_recall_at_k(prediction, ground_truth, k=10):
    """
    Calculates Precision@K and Recall@K for top-K recommendations.
    Parameters:
        prediction (numpy array): The predicted interaction matrix with scores.
        ground_truth (numpy array): The ground truth interaction matrix (binary).
        k (int): Number of top recommendations to consider.
    Returns:
        precision_at_k (float): The average precision@K over all users.
        recall_at_k (float): The average recall@K over all users.
    """
    num_users = prediction.shape[0]
    precision_at_k, recall_at_k = 0, 0

    for user in range(num_users):
        # Step 1: Get the indices of the top-K items for the user based on predicted scores
        # TODO: Use np.argsort to get the top-K items for the current user
        top_k_items = ...  # Complete this line to retrieve top-K item indices for the user
        
        # Step 2: Calculate the number of relevant items in the top-K items for the user
        # TODO: Count how many of the top-K items are actually relevant (i.e., present in ground truth)
        relevant_items_in_top_k = ...  # Complete this line to count relevant items in top-K
        
        # Step 3: Calculate the total number of relevant items for the user
        # TODO: Sum the total relevant items in the ground_truth for this user
        total_relevant_items = ...  # Complete this line to get the total relevant items

        # Update Precision@K and Recall@K for this user
        precision_at_k += relevant_items_in_top_k / k
        recall_at_k += relevant_items_in_top_k / total_relevant_items if total_relevant_items > 0 else 0

    # Step 4: Calculate the average Precision@K and Recall@K over all users
    precision_at_k /= num_users
    recall_at_k /= num_users
    
    return precision_at_k, recall_at_k


After completing the function, you can calculate Precision@K and Recall@K for the user-based and item-based predictions as follows:



In [ ]:
# Run this line after implementing the function
precision_user_k, recall_user_k = precision_recall_at_k(user_prediction, test_data_matrix, k=10)
precision_item_k, recall_item_k = precision_recall_at_k(item_prediction, test_data_matrix, k=10)

print('User-based CF Precision@K:', precision_user_k)
print('User-based CF Recall@K:', recall_user_k)
print('Item-based CF Precision@K:', precision_item_k)
print('Item-based CF Recall@K:', recall_item_k)


## Task 6: Show the Recommendations for a Specific User

Pick a user at random. Extract the top-10 recommendations for this user by both the User-to-User and Item-to-Item Collaborative Filtering models. Present the recommendations by showing the movie posters.

__Hint:__ You can get the movie posters from the TMDb API. In the `links` DataFrame, you’ll find the TMDb ID corresponding to each movie. Using this ID, retrieve the movie poster and display it.

### Step 6.1: Select a User and Get Recommendations

Start by selecting a user at random, and then extract the top-10 recommended movies for that user from both the User-to-User and Item-to-Item models.

In [ ]:
# Your code here

### Step 6.2: Create Recommendation DataFrames with TMDb IDs

Merge the recommendations with the `links` and `movies` DataFrames to create two DataFrames (`user_recommendations_df` and `item_recommendations_df`) with columns `movie_id`, `title`, and `tmdbId`.

In [ ]:
# Your code here

### Fetching and Displaying Movie Posters

The following code uses the **TMDb API** to fetch movie posters based on the `tmdbId` of each recommended movie. The `fetch_poster` function retrieves the movie poster URL, and the `show_recommendations` function displays the posters for the top-10 recommendations. 

You’ll use these functions in the next step to display the movie recommendations for a specific user.

In [ ]:
%pip install tmdbv3api

In [97]:
from tmdbv3api import TMDb, Movie
import pandas as pd

# Set up the TMDb API with your API key
tmdb = TMDb()
tmdb.api_key = '37117cc09d5e00673f14f1cb7c7468b2'  # Replace with your actual TMDb API key

movie_api = Movie()

# Function to fetch movie posters
def fetch_poster(tmdb_id):
    try:
        details = movie_api.details(tmdb_id)
        poster_path = details.poster_path
        return f"https://image.tmdb.org/t/p/w200{poster_path}" if poster_path else None
    except:
        return None

#### Displaying Movie Posters for Top-10 Recommendations

In this step, you’ll complete the `show_recommendations` function to display the movie posters for the top-10 recommendations for a specific user, using the `tmdbId` from the `links` DataFrame.

1. **Fetch the `tmdbId`** for each movie in the recommendations:
   - Use the `links` DataFrame to retrieve the `tmdbId` for each recommended `movie_id`.
   - This `tmdbId` is then used to fetch the poster image via the TMDb API.

2. **Fetch and Display the Poster Image**:
   - Use the `fetch_poster` function (already defined) to get the poster URL based on the `tmdbId`.
   - If the poster URL is available, retrieve the image using the `requests` library and display it in the subplot.
   - If no poster is available, show a "No Image" placeholder text.

Here’s the code with placeholders where you need to complete each step:

In [ ]:
import matplotlib.pyplot as plt
import requests
from PIL import Image
from io import BytesIO

def show_recommendations(recommendations, model_name):
    """
    Displays the movie posters for the top-10 recommended movies.
    
    Parameters:
    - recommendations: List or array of recommended movie IDs.
    - model_name: Name of the recommendation model (e.g., "User-to-User CF" or "Item-to-Item CF").
    """
    fig, axes = plt.subplots(1, len(recommendations), figsize=(20, 5))
    fig.suptitle(f"Top-10 Recommendations for User {user_id} - {model_name}", fontsize=16)

    for i, movie_id in enumerate(recommendations):
        # Step 1: Fetch the `tmdbId` for the movie from the `links` DataFrame
        # TODO: Use `movie_id` to locate the corresponding `tmdbId`
        # TODO: Ensure `tmdb_id` is extracted as an integer
        tmdb_id = ...  # Complete this line to retrieve the `tmdbId`
        
        if len(tmdb_id) == 0:
            axes[i].axis('off')
            continue

        # Step 2: Fetch the poster URL using the `fetch_poster` function
        # TODO: Pass `tmdb_id` to `fetch_poster` and store the result in `poster_url`
        poster_url = ...  # Complete this line to fetch the poster URL
        
        # Step 3: Display the poster image if available
        if poster_url:
            # TODO: Retrieve and display the image
            response = ...  # Complete this line to get the image response
            img = Image.open(BytesIO(response.content))
            axes[i].imshow(img)
            axes[i].axis('off')
        else:
            # Display "No Image" placeholder text if no poster is available
            axes[i].text(0.5, 0.5, "No Image", ha='center', va='center')
            axes[i].axis('off')
    
    plt.show()

# Example usage after completing the TODOs:
# Display top-10 recommendations for User-to-User CF
show_recommendations(user_top_10, "User-to-User CF")

# Display top-10 recommendations for Item-to-Item CF
show_recommendations(item_top_10, "Item-to-Item CF")
